# メルセンヌ素数

In [1]:
import math
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import *
from tqdm import tqdm_notebook as tqdm
from decimal import *
getcontext()
from functools import reduce

## メルセンヌ素数を探す

まずは素数判定関数を定義

In [2]:
def find_a_factor(num):
    for k in range(2, int(math.sqrt(num))+1):
        if num%k == 0:
            return k
            break
    else: return num
    
def is_prime(num):
    return find_a_factor(num)==num

素数リスト `plist` の作成

In [4]:
Num = 60
plist = [n for n in tqdm(range(2,Num+1)) if is_prime(n)]
print(plist)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]


メルセンヌ素数を素朴に探す。

In [5]:
for k in tqdm(plist):
    if is_prime(k): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

2^2-1 = 3 is prime
2^3-1 = 7 is prime
2^5-1 = 31 is prime
2^7-1 = 127 is prime
2^11-1 = 2047 is prime
2^13-1 = 8191 is prime
2^17-1 = 131071 is prime
2^19-1 = 524287 is prime
2^23-1 = 8388607 is prime
2^29-1 = 536870911 is prime
2^31-1 = 2147483647 is prime
2^37-1 = 137438953471 is prime
2^41-1 = 2199023255551 is prime
2^43-1 = 8796093022207 is prime
2^47-1 = 140737488355327 is prime
2^53-1 = 9007199254740991 is prime
2^59-1 = 576460752303423487 is prime



$2^{61}-1$ の素数判定はずいぶん時間がかかる。

In [ ]:
def find_factor_w_tqdm(num):
    for k in tqdm(range(2, int(math.sqrt(num))+1)):
        if num%k == 0:
            return k
            break
    else: return 0

In [ ]:
result = find_factor_w_tqdm(2**61-1); print(result)

## フェルマーテスト

素数判定をもっと高速のものにしたい。まずはフェルマーテスト。

In [6]:
def fermat(num):
    num = abs(num)
    if num == 2:
        return True
    elif num < 2 or num%2 == 0:
        return False
    else:
        return pow(2, num-1, num) == 1

In [7]:
for k in tqdm(plist):
    if fermat(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

2^2-1 = 3 is prime
2^3-1 = 7 is prime
2^5-1 = 31 is prime
2^7-1 = 127 is prime
2^11-1 = 2047 is prime
2^13-1 = 8191 is prime
2^17-1 = 131071 is prime
2^19-1 = 524287 is prime
2^23-1 = 8388607 is prime
2^29-1 = 536870911 is prime
2^31-1 = 2147483647 is prime
2^37-1 = 137438953471 is prime
2^41-1 = 2199023255551 is prime
2^43-1 = 8796093022207 is prime
2^47-1 = 140737488355327 is prime
2^53-1 = 9007199254740991 is prime
2^59-1 = 576460752303423487 is prime



上の出力のように、$2^{11}-1$ や $2^{23}-1$ が素数と判定されてしまった。

## リュカ-レーマー・テスト

#### 疑似コード(from Wikipedia)
```
入力: p:奇素数であるテスト対象の整数
出力: PRIME:素数の場合, COMPOSIT:合成数の場合
Lucas_Lehmer_Test(p):
    var s = 4
    var M = 2**p − 1
    for n in range(2, p):
        s = (s*s-2)%M
    if s == 0 then
        return PRIME
    else
        return COMPOSIT
```

In [8]:
def lucas_lehmer(p):
    s = 4
    mp = 2**p-1
    for n in range(p-2): # p-2 times iteration (list(range(p-2))=[0,1,...,p-1])
         s = (s**2-2)%mp
    return s==0

In [9]:
def lucas_lehmer_FAST(p):
    s = 4
    mp = (1<<p)-1
    for n in range(p-2):
        ss = s*s        
        s = (ss & mp) + (ss >> p)
        if s >= mp: s = s-mp
        s = s-2
    return s==0

In [10]:
%timeit lucas_lehmer(4423)
%timeit lucas_lehmer_FAST(4423)

1 loop, best of 3: 174 ms per loop
10 loops, best of 3: 50 ms per loop


### リュカ-レーマー・テストを使う。

In [11]:
def mp_str(p):
    mp = Decimal(2**p-1)
    nod = mp.adjusted()+1
    if nod <= 40:
        return str(mp)
    else:
        getcontext().prec = nod
        top = (mp/(Decimal(10)**(nod-20))).quantize(1)
        bottom = mp-math.floor(mp/Decimal(1.0e+20))*Decimal(1.0e+20)
        return '{}...{} [{} digits]'.format(top,bottom,nod)

def find_mp(nmin, nmax):
    plist = [n for n in range(nmin, nmax+1) if is_prime(n)]
    for p in tqdm(plist):
        if lucas_lehmer_FAST(p): print('2^{}-1 = {} is prime'.format(p, mp_str(p)))

In [12]:
find_mp(100, 1000)

2^107-1 = 162259276829213363391578010288127 is prime
2^127-1 = 170141183460469231731687303715884105727 is prime
2^521-1 = 68647976601306097150...12574028291115057151 [157 digits] is prime
2^607-1 = 53113799281676709869...70835393219031728127 [183 digits] is prime



In [ ]:
find_mp(1000, 4000)

In [ ]:
find_mp(10000, 20000)

In [ ]:
find_mp(20000, 50000)

### リュカ-レーマー・テストのコードの改良

$s_{p-2}$ を一般項から直接求める。

In [ ]:
getcontext().prec=100000
p = 17
mp = 2**p-1
seq = [4]
for n in range(p-2): seq.append((seq[-1]**2-2)%mp)
print(seq)

In [ ]:
def binom(n,k,mod):
#    nume = Decimal(math.factorial(n))
#    deno = Decimal(math.factorial(k)*math.factorial(n-k))
    if k==0: nume, deno = 1, 1
    else:
        nume = reduce(lambda x,y: x*y, range(n-k+1, n+1))
        deno = reduce(lambda x,y: x*y, range(1, k+1))
    return Decimal(nume)/Decimal(deno)

In [ ]:
getcontext().prec=10000
p = 17
mp = Decimal(2**p-1)
for i in range(1,p-2+1):
    n = 2**i
    seq = [2*binom(n,2*k,mp)*2**(n-2*k)*3**k for k in range(2**(i-1)+1)]
    print(sum(seq)%mp)

In [ ]:
p = 17
mp = (1<<p)-1
n = 1<<(p-2)
sum([2*binom(n,2*k,mp)*(2**(n-2*k))*(3**k) for k in range(int(n/2)+1)])%mp

### "メルセンヌ合成数" の素因数分解

In [13]:
getcontext().prec=10000
def factorisation(num, facs_list=None):
    if facs_list is None: facs_list = []
    facs_list.append(find_a_factor(num))
    if facs_list[-1]!=num:
        num = int(Decimal(num)/Decimal(facs_list[-1]))
        facs_list=factorisation(num, facs_list)
    return facs_list

In [14]:
N = 80
plist = [n for n in list(range(3, N)) if is_prime(n)]

for p in plist:
    if not lucas_lehmer_FAST(p):
        mp = (1<<p)-1
        fctd = "*".join(map(str,factorisation(mp)))
        print("2**{}-1={}={}".format(p,mp,fctd))

2**11-1=2047=23*89
2**23-1=8388607=47*178481
2**29-1=536870911=233*1103*2089
2**37-1=137438953471=223*616318177
2**41-1=2199023255551=13367*164511353
2**43-1=8796093022207=431*9719*2099863
2**47-1=140737488355327=2351*4513*13264529
2**53-1=9007199254740991=6361*69431*20394401
2**59-1=576460752303423487=179951*3203431780337
2**67-1=147573952589676412927=193707721*761838257287
2**71-1=2361183241434822606847=228479*48544121*212885833
2**73-1=9444732965739290427391=439*2298041*9361973132609
2**79-1=604462909807314587353087=2687*202029703*1113491139767


In [15]:
factorisation(2**83-1)

[167, 57912614113275649087721]

In [19]:
factorisation(Decimal(2**83-1)/167+1)

[2, 3, 3, 3, 3, 23, 23, 59, 373, 30707300495827]

## ミラーラビン法

確率的判定法。

In [ ]:
import random
def millerrabin(n):
    if n%2 == 0 or n%3 == 0 or n%5 == 0:
        return False
    else:
        s, d = 0, n-1
        while d%2==0: s,d = s+1, int(d/2)
        k = 50
#        for j in tqdm(range(k)):
        while k > 0:
            k = k-1
            a = random.randint(1,n-1)
            t, y = d, pow(a,d,n)
            while t != n-1 and y != 1 and y != n-1:
                y = pow(y,2,n)
                t <<= 1
            if y != n-1 and t%2 == 0:
                return False
        return True

In [ ]:
for k in tqdm(plist):
    if millerrabin(2**k-1): print('2^{}-1 = {} is prime'.format(k, 2**k-1))

ずいぶん時間がかかってしまう。なぜ?